# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1) A two-sided z-test is most appropriate here, as our sample sizes are sufficiently large, observations are independent and random. Due to the samples sizes, the CLT also applies.

2) H(0): There is no significant difference in the callback rate between resumes with 'white-sounding' and 'black-sounding' names.

H(a): There is a significant difference in the callback rate between resumes with 'white-sounding' and 'black-sounding' names.

In [11]:
w = data[data.race=='w']
b = data[data.race=='b']
mw = np.mean(w.call)
mb = np.mean(b.call) 
vw = np.var(w.call)
vb = np.var(b.call)

In [26]:
# Your solution to Q3 here
#Z-test
z = (mw-mb)/np.sqrt((vw/len(w))+(vb/len(b)))
p_val = stats.norm.sf(z)*2
print('p-value: '+str(p_val))
moe = z*np.sqrt((vw/len(w))+(vb/len(b)))
print('Margin of Error: '+str(moe))
ci= (mw-mb)+np.array([-1,1])*np.sqrt((vw/len(w))+(vb/len(b)))*1.96
print ('Confidence Interval (95%): '+str(ci))


p-value: 3.8620110799840656e-05
Margin of Error: 0.03203285485506058
Confidence Interval (95%): [0.01677757 0.04728814]


The average call back rate for resumes with 'white-sounding' names is between 1.6% and 4.7% higher than resumes with 'black-sounding' names, at a 95% confidence level.

In [30]:
#Bootstrapping
diff_means = np.empty(100000)
for i in range(100000):
    d = np.random.permutation(np.concatenate((w.call,b.call)))
    w_bs = d[:len(w)]
    b_bs = d[len(w):]
    diff_means[i] = abs(np.mean(w_bs)-np.mean(b_bs))

p_val = np.sum(diff_means > (mw-mb))/len(np.concatenate((w,b)))

In [35]:
p_val
#P-value far below 5% threshhold

0.0006160164271047228

Based on the p-values from bootstrapping and z-testing, we reject the null hypothesis.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

4) Based on the p-values from bootstrapping and z-testing, we reject the null hypothesis. There is a significant difference between average callback rate for resumes with 'black-sounding' vs. 'white-sounding' names. The average call back rate for resumes with 'white-sounding' names is between 1.6% and 4.7% higher than resumes with 'black-sounding' names, at a 95% confidence level.

5) While name does have a significant impact, that does not mean it is the only, or most impactful, factor is getting a call back. Future analysis could look at years of experience, educations, and gender as additional factors.

In [34]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')